In [1]:
import struct
import numpy as np

In [2]:
def read_hair_file(file):
    with open(file, mode='rb') as f:
        num_strand = f.read(4)
        (num_strand,) = struct.unpack('I', num_strand)
        point_count = f.read(4)
        (point_count,) = struct.unpack('I', point_count)

        segments = f.read(2 * num_strand)
        segments = struct.unpack('H' * num_strand, segments)
        segments = list(segments)
        num_points = sum(segments)

        points = f.read(4 * num_points * 3)
        points = struct.unpack('f' * num_points * 3, points)
    
    points = list(points)
    points = np.array(points)
    points = np.reshape(points, (-1, 3))

    segments_with_direction = []
    index = 0
    
    for strand_length in segments:
        strand_points = points[index:index + strand_length]
        directions = []
        
        if len(strand_points) > 1:
            dx = strand_points[1][0] - strand_points[0][0]
            dy = strand_points[1][1] - strand_points[0][1]
            dz = strand_points[1][2] - strand_points[0][2]
            directions.append((dx, dy, dz))  

            for i in range(1, len(strand_points)):
                dx = strand_points[i][0] - strand_points[i-1][0]
                dy = strand_points[i][1] - strand_points[i-1][1]
                dz = strand_points[i][2] - strand_points[i-1][2]
                directions.append((dx, dy, dz))
        else:
            directions.append((0.0, 0.0, 0.0))

        strand_with_direction = []
        for point, (dx, dy, dz) in zip(strand_points, directions):
            strand_with_direction.append((*point, dx, dy, dz))
        
        segments_with_direction.append(strand_with_direction)
        index += strand_length
    
    return segments_with_direction

def write_ply_file(filename, segments_with_direction):
    with open(filename, 'w') as f:
        num_vertices = sum(len(segment) for segment in segments_with_direction)
        f.write(f"ply\n")
        f.write(f"format ascii 1.0\n")
        f.write(f"element vertex {num_vertices}\n")
        f.write(f"property float x\n")
        f.write(f"property float y\n")
        f.write(f"property float z\n")
        f.write(f"property float dx\n")
        f.write(f"property float dy\n")
        f.write(f"property float dz\n")
        f.write(f"end_header\n")
        
        for strand in segments_with_direction:
            for point in strand:
                f.write(f"{point[0]} {point[1]} {point[2]} {point[3]} {point[4]} {point[5]}\n")


In [51]:
file = '/home/sharma/MonoHair/data/ct2wavy/output/10-16/full/connected_strands.hair'
segments_with_direction = read_hair_file(file)
write_ply_file('/home/sharma/MonoHair/data/ct2wavy/result.ply', segments_with_direction)

In [52]:
def read_ply(file_path):
    # Read binary .ply file
    with open(file_path, 'rb') as file:
        header = ""
        # Read the header
        while True:
            line = file.readline().decode('utf-8').strip()
            header += line + "\n"
            if line.startswith("end_header"):
                break
        
        # Extract vertex count from the header
        vertex_count = int([line.split()[2] for line in header.splitlines() if line.startswith("element vertex")][0])
        
        # Read the vertex data (only x, y, z)
        vertices = []
        for _ in range(vertex_count):
            data = struct.unpack('<3f4B', file.read(3*4 + 4))  # Read x, y, z, red, green, blue, alpha
            x, y, z = data[:3]  # Only x, y, z are used
            vertices.append((x, y, z))
        
        return header, vertices
    
def transform_vertices(vertices, rotation_matrix, translation_vector):
    """ Apply rotation and translation to a list of vertices. """
    vertices_np = np.array(vertices)  # (N, 3)
    
    # Apply rotation
    rotated = vertices_np @ rotation_matrix.T  # (N, 3)
    
    # Apply translation
    translated = rotated + translation_vector  # (N, 3)
    
    return [tuple(v) for v in translated]

def write_ply(file_path, header, vertices):
    # Modify the header:
    lines = header.strip().split('\n')
    new_header_lines = []
    for line in lines:
        if line.startswith("format"):
            new_header_lines.append("format ascii 1.0")  # Change format to ASCII
        elif line.startswith("end_header"):
            # Insert new properties before end_header
            new_header_lines.append("property float dx")
            new_header_lines.append("property float dy")
            new_header_lines.append("property float dz")
            new_header_lines.append("end_header")
        else:
            new_header_lines.append(line)
    
    new_header = "\n".join(new_header_lines) + "\n"

    with open(file_path, 'w') as file:
        # Write the updated header
        file.write(new_header)

        # Write the vertex data
        for i in range(len(vertices)):
            x, y, z = vertices[i]
            
            if i == 0:
                dx, dy, dz = vertices[1][0] - x, vertices[1][1] - y, vertices[1][2] - z
            elif (i + 1) % 100 == 0:
                if i + 1 < len(vertices):
                    dx, dy, dz = vertices[i+1][0] - x, vertices[i+1][1] - y, vertices[i+1][2] - z
                else:
                    dx, dy, dz = vertices[i][0] - vertices[i-1][0], vertices[i][1] - vertices[i-1][1], vertices[i][2] - vertices[i-1][2]
            else:
                prev_x, prev_y, prev_z = vertices[i - 1]
                dx, dy, dz = x - prev_x, y - prev_y, z - prev_z
            
            file.write(f"{x} {y} {z} {dx} {dy} {dz}\n")


def calculate_dx_dy_dz(input_ply, output_ply):
    header, vertices = read_ply(input_ply)
    translation_vector = np.array([-0.01, 1.654, -0.27])
    theta_y = np.deg2rad(180)
    theta_x = np.deg2rad(105)

    R_y = np.array([
        [np.cos(theta_y), 0, np.sin(theta_y)],
        [0, 1, 0],
        [-np.sin(theta_y), 0, np.cos(theta_y)]
    ])

    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(theta_x), -np.sin(theta_x)],
        [0, np.sin(theta_x), np.cos(theta_x)]
    ])

    rotation_matrix = R_x @ R_y
    print(len(vertices))
    vertices = transform_vertices(vertices,rotation_matrix,translation_vector)
    print(len(vertices))
    write_ply(output_ply, header, vertices)

In [53]:
input_ply = "/home/sharma/Downloads/ct2hairpcs/Wavy.ply" 
output_ply = "ground_truth_wavy.ply"
calculate_dx_dy_dz(input_ply, output_ply)

10262300
10262300


In [45]:
import torch
import numpy as np
import open3d as o3d
from plyfile import PlyData
from scipy.spatial import cKDTree

def read_ply(ply_file):
    
    ply_data = PlyData.read(ply_file) 
    
    x = np.array(ply_data['vertex']['x'])
    y = np.array(ply_data['vertex']['y'])
    z = np.array(ply_data['vertex']['z'])
    dx = np.array(ply_data['vertex']['dx'])
    dy = np.array(ply_data['vertex']['dy'])
    dz = np.array(ply_data['vertex']['dz'])
    
    full_data = np.column_stack((x, y, z, dx, dy, dz))
    
    return torch.tensor(full_data, dtype=torch.float32)

def compute_angular_distance(d1, d2):
    d1_norm = d1 / torch.norm(d1)
    d2_norm = d2 / torch.norm(d2)
    
    cos_theta = torch.clamp(torch.dot(d1_norm, d2_norm), -1.0, 1.0)
    return torch.acos(cos_theta) * 180.0 / torch.pi


def compute_metrics(p1, p2, threshold_euclidean, threshold_angular):
    
    if hasattr(p1, 'numpy'):
        p1_np = p1.detach().cpu().numpy()
    else:
        p1_np = p1
    
    if hasattr(p2, 'numpy'):
        p2_np = p2.detach().cpu().numpy()
    else:
        p2_np = p2
    
    tree = cKDTree(p2_np[:, :3])
    
    tp, fp = 0, 0
    matched_indices = set()
    
    for i in range(p1_np.shape[0]):
        neighbors = tree.query_ball_point(p1_np[i, :3], r=threshold_euclidean)
        
        if not neighbors:
            fp += 1
            continue
        
        found_match = False
        
        for idx in neighbors:
            if idx in matched_indices:
                continue
            
            # Compute cosine similarity faster
            v1 = p1_np[i, 3:]
            v2 = p2_np[idx, 3:]
            cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            cos_sim = np.clip(cos_sim, -1.0, 1.0) 
            angular_dist = np.arccos(cos_sim)
            
            if angular_dist <= threshold_angular:
                tp += 1
                matched_indices.add(idx)
                found_match = True
                break
        
        if not found_match:
            fp += 1
    
    fn = len(p2_np) - len(matched_indices)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * recall * precision / (recall + precision) if (recall + precision) > 0 else 0
    
    return tp, fp, fn, precision, recall, f1_score

def main(ground_truth_ply, results_ply, threshold_euclidean, threshold_angular):
    
    p1 = read_ply(results_ply)  # Regenerated 
    p2 = read_ply(ground_truth_ply)  # Original
    print(p1.shape)
    print(p2.shape)
    
    tp, fp, fn, precision, recall, f1_score = compute_metrics(p1, p2, threshold_euclidean, threshold_angular)
    
    print(f"True Positives (TP): {tp}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1_score}")


In [55]:
ground_truth_ply = "ground_truth_wavy.ply"
results_ply = "/home/sharma/MonoHair/data/ct2wavy/result.ply"


threshold_euclidean = 0.002
threshold_angular = 20

main(ground_truth_ply, results_ply, threshold_euclidean, threshold_angular)

AttributeError: 'tuple' object has no attribute 'shape'